In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('/content/diabetes_data_upload.csv')

In [3]:
df

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,39,Female,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Yes,Yes,No,No,No,Positive
516,48,Female,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,No,No,No,Positive
517,58,Female,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,Yes,No,Yes,Positive
518,32,Female,No,No,No,Yes,No,No,Yes,Yes,No,Yes,No,No,Yes,No,Negative


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

X = df.drop('class', axis=1)
y = df['class']

# Convert categorical features to numerical using one-hot encoding
X = pd.get_dummies(X)

y = y.map({'Negative': 0, 'Positive': 1})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Train Logistic Regression
logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)
logreg_pred = logreg_model.predict(X_test)


# Train Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
dt_pred = dt_model.predict(X_test)


# Train Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)


# Evaluate the models
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    false_negative_rate = fn / (fn + tp)
    return accuracy, precision, recall, false_negative_rate

logreg_accuracy, logreg_precision, logreg_recall, logreg_fnr = evaluate_model(y_test, logreg_pred)
dt_accuracy, dt_precision, dt_recall, dt_fnr = evaluate_model(y_test, dt_pred)
rf_accuracy, rf_precision, rf_recall, rf_fnr = evaluate_model(y_test, rf_pred)


print("Logistic Regression:")
print("Accuracy:", logreg_accuracy)
print("Precision:", logreg_precision)
print("Recall:", logreg_recall)
print("False Negative Rate:", logreg_fnr)
print()

print("Decision Tree:")
print("Accuracy:", dt_accuracy)
print("Precision:", dt_precision)
print("Recall:", dt_recall)
print("False Negative Rate:", dt_fnr)
print()

print("Random Forest:")
print("Accuracy:", rf_accuracy)
print("Precision:", rf_precision)
print("Recall:", rf_recall)
print("False Negative Rate:", rf_fnr)
print()


# Identify the model with the lowest false negative rate
models = {
    'Logistic Regression': logreg_fnr,
    'Decision Tree': dt_fnr,
    'Random Forest': rf_fnr
}
best_model = min(models, key=models.get)
print(f"The model with the lowest false negative rate is: {best_model}")



Logistic Regression:
Accuracy: 0.9230769230769231
Precision: 0.9315068493150684
Recall: 0.9577464788732394
False Negative Rate: 0.04225352112676056

Decision Tree:
Accuracy: 0.9615384615384616
Precision: 0.9855072463768116
Recall: 0.9577464788732394
False Negative Rate: 0.04225352112676056

Random Forest:
Accuracy: 0.9903846153846154
Precision: 1.0
Recall: 0.9859154929577465
False Negative Rate: 0.014084507042253521

The model with the lowest false negative rate is: Random Forest


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
from scipy import stats

results_df = pd.DataFrame({'Age': df.iloc[y_test.index]['Age'],
                           'True Label': y_test,
                           'Predicted Label': logreg_pred})

correctly_classified_diabetic = results_df[(results_df['True Label'] == 1) & (results_df['Predicted Label'] == 1)]['Age']
misclassified_diabetic = results_df[(results_df['True Label'] == 1) & (results_df['Predicted Label'] == 0)]['Age']

t_statistic, p_value = stats.ttest_ind(correctly_classified_diabetic, misclassified_diabetic)

print(f"T-statistic: {t_statistic}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print("There is a significant difference in mean age between correctly and misclassified diabetic patients.")
    print("Age might be a relevant feature for predicting diabetes.")
else:
    print("There is no significant difference in mean age between correctly and misclassified diabetic patients.")
    print("Age might not be a strong predictor of diabetes in this context.")

print("\n")
if logreg_fnr > dt_fnr or logreg_fnr > rf_fnr:
  print("Consider using a different model like Decision Tree or Random Forest for better classification, especially if minimizing false negatives is a priority.")
else:
  print("The logistic regression model is performing well.")

T-statistic: 1.3175831097449258
P-value: 0.19200148354386326
There is no significant difference in mean age between correctly and misclassified diabetic patients.
Age might not be a strong predictor of diabetes in this context.


Consider using a different model like Decision Tree or Random Forest for better classification, especially if minimizing false negatives is a priority.


In [10]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from scipy import stats

# Evaluate the models for false positive rate
def evaluate_model_fpr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0  # Handle division by zero
    return false_positive_rate

rf_fpr = evaluate_model_fpr(y_test, rf_pred)
print("Random Forest False Positive Rate:", rf_fpr)


# One-Sample Z-Test for Random Forest
if rf_fpr > 0.20:
  z_statistic, p_value = stats.ztest(
      [rf_fpr], value=0.20, alternative='larger'
  )
  print("One-Sample Z-Test for Random Forest FPR:")
  print("Z-statistic:", z_statistic)
  print("P-value:", p_value)
  if p_value < 0.05:
      print("The false positive rate is significantly higher than 20%.")


# Gradient Boosting Model
gb_model = GradientBoostingClassifier()
gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_test)
gb_fpr = evaluate_model_fpr(y_test, gb_pred)
print("\nGradient Boosting False Positive Rate:", gb_fpr)

# Compare models
if gb_fpr < rf_fpr:
  print("Gradient Boosting Model performs better in reducing Type I errors compared to Random Forest.")
else:
  print("Random Forest Model performs better in reducing Type I errors compared to Gradient Boosting.")

Random Forest False Positive Rate: 0.0

Gradient Boosting False Positive Rate: 0.0
Random Forest Model performs better in reducing Type I errors compared to Gradient Boosting.


In [13]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


# Train SVM
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)


# Train KNN
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
knn_pred = knn_model.predict(X_test)


# Evaluate SVM
svm_accuracy, svm_precision, svm_recall, svm_fnr = evaluate_model(y_test, svm_pred)
print("SVM:")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)
print("False Negative Rate:", svm_fnr)
print()

# Evaluate KNN
knn_accuracy, knn_precision, knn_recall, knn_fnr = evaluate_model(y_test, knn_pred)
print("KNN:")
print("Accuracy:", knn_accuracy)
print("Precision:", knn_precision)
print("Recall:", knn_recall)
print("False Negative Rate:", knn_fnr)
print()

from statsmodels.stats.proportion import proportions_ztest

# Get the number of successes (false negatives) and trials for each model
svm_fn = int(svm_fnr * len(y_test))  # Number of false negatives for SVM
knn_fn = int(knn_fnr * len(y_test))  # Number of false negatives for KNN
nobs = [len(y_test), len(y_test)]  # Number of trials for each model

z_statistic, p_value = proportions_ztest([svm_fn, knn_fn], nobs)

print("Z-Test between SVM and KNN FNR:")
print("Z-statistic:", z_statistic)
print("P-value:", p_value)

if p_value < 0.05:
    print("The false negative rates of SVM and KNN are significantly different.")
else:
    print("The false negative rates of SVM and KNN are not significantly different.")

print("Z-Test between SVM and KNN FNR:")
print("Z-statistic:", z_statistic)
print("P-value:", p_value)

if p_value < 0.05:
    print("The false negative rates of SVM and KNN are significantly different.")
else:
    print("The false negative rates of SVM and KNN are not significantly different.")


# Compare models and find the one with the lowest FNR
models_fnr = {
    "Logistic Regression": logreg_fnr,
    "SVM": svm_fnr,
    "KNN": knn_fnr,
    "Decision Tree": dt_fnr,
    "Random Forest": rf_fnr,
}

best_model_fnr = min(models_fnr, key=models_fnr.get)
print(
    f"The model with the lowest false negative rate is: {best_model_fnr} with a FNR of {models_fnr[best_model_fnr]}"
)

# Recommendation for real-world deployment
print("\nRecommendation for real-world deployment:")

if best_model_fnr == "Logistic Regression" or best_model_fnr == "SVM" or best_model_fnr == "KNN":
    print(
        f"Based on the results, the {best_model_fnr} model is recommended for real-world deployment as it minimizes the risk of undiagnosed diabetes cases."
    )
else:
    print(
        f"Based on the results, the {best_model_fnr} model is recommended for real-world deployment as it minimizes the risk of undiagnosed diabetes cases."
    )


SVM:
Accuracy: 0.6826923076923077
Precision: 0.6826923076923077
Recall: 1.0
False Negative Rate: 0.0

KNN:
Accuracy: 0.875
Precision: 0.967741935483871
Recall: 0.8450704225352113
False Negative Rate: 0.15492957746478872

Z-Test between SVM and KNN FNR:
Z-statistic: -4.163331998932266
P-value: 3.1363681444258854e-05
The false negative rates of SVM and KNN are significantly different.
Z-Test between SVM and KNN FNR:
Z-statistic: -4.163331998932266
P-value: 3.1363681444258854e-05
The false negative rates of SVM and KNN are significantly different.
The model with the lowest false negative rate is: SVM with a FNR of 0.0

Recommendation for real-world deployment:
Based on the results, the SVM model is recommended for real-world deployment as it minimizes the risk of undiagnosed diabetes cases.


In [15]:
# Train Gradient Boosting Model
gb_model = GradientBoostingClassifier()
gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_test)

# Evaluate Gradient Boosting Model
gb_accuracy, gb_precision, gb_recall, gb_fnr = evaluate_model(y_test, gb_pred)
print("Gradient Boosting:")
print("Accuracy:", gb_accuracy)
print("Precision:", gb_precision)
print("Recall:", gb_recall)
print("False Negative Rate:", gb_fnr)
print()

# Misclassification Analysis
results_df = pd.DataFrame({
    'Age': df.iloc[y_test.index]['Age'],
    'True Label': y_test,
    'Predicted Label': gb_pred
})

correctly_classified_diabetic = results_df[
    (results_df['True Label'] == 1) & (results_df['Predicted Label'] == 1)]['Age']
misclassified_diabetic = results_df[
    (results_df['True Label'] == 1) & (results_df['Predicted Label'] == 0)]['Age']

# Z-Test
t_statistic, p_value = stats.ttest_ind(correctly_classified_diabetic,
                                      misclassified_diabetic)

print(f"T-statistic: {t_statistic}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print(
        "There is a significant difference in mean age between correctly and misclassified diabetic patients."
    )
    print("Age might be a relevant feature for predicting diabetes.")
else:
    print(
        "There is no significant difference in mean age between correctly and misclassified diabetic patients."
    )
    print("Age might not be a strong predictor of diabetes in this context.")

# Adjust the model to reduce Type II errors (if significant)
if p_value < 0.05:
    print(
        "\nTo reduce Type II errors (false negatives), consider increasing the model's sensitivity."
    )
    print(
        "You can try adjusting the threshold for classification or exploring techniques like cost-sensitive learning to emphasize the cost of misclassifying diabetic patients."
    )


# Compare with Random Forest Model
print("\nComparing Gradient Boosting and Random Forest:")
print("Gradient Boosting False Negative Rate:", gb_fnr)
print("Random Forest False Negative Rate:", rf_fnr)

if gb_fnr < rf_fnr:
    print(
        "Gradient Boosting Model has fewer Type II errors compared to Random Forest."
    )
    print(
        "Gradient Boosting is preferable for medical use, especially if minimizing false negatives is critical."
    )
else:
    print(
        "Random Forest Model has fewer Type II errors compared to Gradient Boosting."
    )
    print(
        "Random Forest is preferable for medical use, especially if minimizing false negatives is critical."
    )


Gradient Boosting:
Accuracy: 0.9711538461538461
Precision: 1.0
Recall: 0.9577464788732394
False Negative Rate: 0.04225352112676056

T-statistic: 1.8205139869779916
P-value: 0.07301728982680411
There is no significant difference in mean age between correctly and misclassified diabetic patients.
Age might not be a strong predictor of diabetes in this context.

Comparing Gradient Boosting and Random Forest:
Gradient Boosting False Negative Rate: 0.04225352112676056
Random Forest False Negative Rate: 0.014084507042253521
Random Forest Model has fewer Type II errors compared to Gradient Boosting.
Random Forest is preferable for medical use, especially if minimizing false negatives is critical.


In [16]:

# Train SVM
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)

# Train KNN
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
knn_pred = knn_model.predict(X_test)

# Evaluate SVM
svm_accuracy, svm_precision, svm_recall, svm_fnr = evaluate_model(y_test, svm_pred)
print("SVM:")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)
print("False Negative Rate:", svm_fnr)
print()

# Evaluate KNN
knn_accuracy, knn_precision, knn_recall, knn_fnr = evaluate_model(y_test, knn_pred)
print("KNN:")
print("Accuracy:", knn_accuracy)
print("Precision:", knn_precision)
print("Recall:", knn_recall)
print("False Negative Rate:", knn_fnr)
print()

# Get the number of successes (false negatives) and trials for each model
svm_fn = int(svm_fnr * len(y_test))  # Number of false negatives for SVM
knn_fn = int(knn_fnr * len(y_test))  # Number of false negatives for KNN
nobs = [len(y_test), len(y_test)]  # Number of trials for each model

z_statistic, p_value = proportions_ztest([svm_fn, knn_fn], nobs)

print("Z-Test between SVM and KNN FNR:")
print("Z-statistic:", z_statistic)
print("P-value:", p_value)

if p_value < 0.05:
    print("The false negative rates of SVM and KNN are significantly different.")
else:
    print("The false negative rates of SVM and KNN are not significantly different.")

# Compare models and find the one with the lowest FNR
models_fnr = {
    "Logistic Regression": logreg_fnr,
    "SVM": svm_fnr,
    "KNN": knn_fnr,
    "Decision Tree": dt_fnr,
    "Random Forest": rf_fnr,
}

best_model_fnr = min(models_fnr, key=models_fnr.get)
print(
    f"The model with the lowest false negative rate is: {best_model_fnr} with a FNR of {models_fnr[best_model_fnr]}"
)

# Recommendation for real-world deployment
print("\nRecommendation for real-world deployment:")

if best_model_fnr == "Logistic Regression" or best_model_fnr == "SVM" or best_model_fnr == "KNN":
    print(
        f"Based on the results, the {best_model_fnr} model is recommended for real-world deployment as it minimizes the risk of undiagnosed diabetes cases."
    )
else:
    print(
        f"Based on the results, the {best_model_fnr} model is recommended for real-world deployment as it minimizes the risk of undiagnosed diabetes cases."
    )

# Train Gradient Boosting Model
gb_model = GradientBoostingClassifier()
gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_test)

# Evaluate Gradient Boosting Model
gb_accuracy, gb_precision, gb_recall, gb_fnr = evaluate_model(y_test, gb_pred)
print("Gradient Boosting:")
print("Accuracy:", gb_accuracy)
print("Precision:", gb_precision)
print("Recall:", gb_recall)
print("False Negative Rate:", gb_fnr)
print()

# Misclassification Analysis
results_df = pd.DataFrame({
    'Age': df.iloc[y_test.index]['Age'],
    'True Label': y_test,
    'Predicted Label': gb_pred
})

correctly_classified_diabetic = results_df[
    (results_df['True Label'] == 1) & (results_df['Predicted Label'] == 1)]['Age']
misclassified_diabetic = results_df[
    (results_df['True Label'] == 1) & (results_df['Predicted Label'] == 0)]['Age']

# Z-Test
t_statistic, p_value = stats.ttest_ind(correctly_classified_diabetic,
                                      misclassified_diabetic)

print(f"T-statistic: {t_statistic}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print(
        "There is a significant difference in mean age between correctly and misclassified diabetic patients."
    )
    print("Age might be a relevant feature for predicting diabetes.")
else:
    print(
        "There is no significant difference in mean age between correctly and misclassified diabetic patients."
    )
    print("Age might not be a strong predictor of diabetes in this context.")

# Adjust the model to reduce Type II errors (if significant)
if p_value < 0.05:
    print(
        "\nTo reduce Type II errors (false negatives), consider increasing the model's sensitivity."
    )
    print(
        "You can try adjusting the threshold for classification or exploring techniques like cost-sensitive learning to emphasize the cost of misclassifying diabetic patients."
    )

# Compare with Random Forest Model
print("\nComparing Gradient Boosting and Random Forest:")
print("Gradient Boosting False Negative Rate:", gb_fnr)
print("Random Forest False Negative Rate:", rf_fnr)

if gb_fnr < rf_fnr:
    print(
        "Gradient Boosting Model has fewer Type II errors compared to Random Forest."
    )
    print(
        "Gradient Boosting is preferable for medical use, especially if minimizing false negatives is critical."
    )
else:
    print(
        "Random Forest Model has fewer Type II errors compared to Gradient Boosting."
    )
    print(
        "Random Forest is preferable for medical use, especially if minimizing false negatives is critical."
    )


SVM:
Accuracy: 0.6826923076923077
Precision: 0.6826923076923077
Recall: 1.0
False Negative Rate: 0.0

KNN:
Accuracy: 0.875
Precision: 0.967741935483871
Recall: 0.8450704225352113
False Negative Rate: 0.15492957746478872

Z-Test between SVM and KNN FNR:
Z-statistic: -4.163331998932266
P-value: 3.1363681444258854e-05
The false negative rates of SVM and KNN are significantly different.
The model with the lowest false negative rate is: SVM with a FNR of 0.0

Recommendation for real-world deployment:
Based on the results, the SVM model is recommended for real-world deployment as it minimizes the risk of undiagnosed diabetes cases.
Gradient Boosting:
Accuracy: 0.9711538461538461
Precision: 1.0
Recall: 0.9577464788732394
False Negative Rate: 0.04225352112676056

T-statistic: 1.8205139869779916
P-value: 0.07301728982680411
There is no significant difference in mean age between correctly and misclassified diabetic patients.
Age might not be a strong predictor of diabetes in this context.

Compa